In [1]:

import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageGrab


In [28]:
def pil2cv(image, transform=cv2.COLOR_BGR2GRAY):
    ''' PIL型 -> OpenCV型
    cv2.COLOR_BGR2GRAY 将BGR格式转换成灰度图片
    cv2.COLOR_BGR2RGB 将BGR格式转换成RGB格式 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:
        pass
    elif new_image.shape[2] == 3:
        new_image = cv2.cvtColor(new_image, transform)
    elif new_image.shape[2] == 4:
        new_image = cv2.cvtColor(new_image, transform)
    return new_image

def cv2pil(image, transform=cv2.COLOR_BGR2GRAY):
    ''' OpenCV型 -> PIL型 
    cv2.COLOR_BGR2GRAY 将BGR格式转换成灰度图片
    cv2.COLOR_BGR2RGB 将BGR格式转换成RGB格式 '''
    new_image = image.copy()
    if new_image.ndim == 2:
        pass
    elif new_image.shape[2] == 3:
        new_image = cv2.cvtColor(new_image, transform)
    elif new_image.shape[2] == 4:
        new_image = cv2.cvtColor(new_image, transform)
    new_image = Image.fromarray(new_image)
    return new_image

In [2]:
IMAGE_TEMPLATE_ONE = 'template_one.jpg'
IMAGE_TEMPLATE_MUL = 'template_mul.jpg'
IMAGE_TEMPLATE_NONE = 'template_none.jpg'
IMAGE_TARGET = 'target.jpg'



In [33]:
# 查找图片A是否在图片B里面出现，A为模板，B为目标文件
# template = cv2.imread(IMAGE_TEMPLATE_ONE, 0)  # 读出来是rbg类型
template = cv2.imread(IMAGE_TEMPLATE_NONE, 0)  # 读出来是rbg类型
target = cv2.imread(IMAGE_TARGET, 0)

# cv2pil(template).show()
# cv2pil(target).show()

w, h = template.shape[::-1]
# print(w, h)

# 单匹配可以看到cv2.TM_CCORR的结果并不如预期那样好
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
           'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
for meth in methods:
    img = target.copy()
    method = eval(meth)

    # 应用模板匹配
    res = cv2.matchTemplate(img, template, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # 如果方法是cv2.TM_SQDIFF或cv2.TM_SQDIFF_NORMED
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    
    bottom_right = (top_left[0] + w, top_left[1] + h)
    print(top_left, bottom_right)
    # cv2.rectangle(img, top_left, bottom_right, 255, 2)
    # plt.subplot(121), plt.imshow(res, 'gray')
    # plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    # plt.subplot(122), plt.imshow(img, 'gray')
    # plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    # plt.suptitle(meth)
    # plt.show()


(307, 359) (355, 407)
(143, 358) (191, 406)
(328, 114) (376, 162)
(89, 359) (137, 407)
(235, 25) (283, 73)
(267, 16) (315, 64)


In [3]:
template = cv2.imread(IMAGE_TEMPLATE_MUL, 0)  # numpy.ndarray
# template = cv2.imread(IMAGE_TEMPLATE_NONE, 0)
# template = cv2.imread(IMAGE_TEMPLATE_ONE, 0)
# target_rgb = cv2.imread('all.jpg', 0)  # 可能报错 Invalid number of channels in input image
target_rgb = cv2.imread(IMAGE_TARGET)
target_rgb_gray = cv2.cvtColor(target_rgb, cv2.COLOR_BGR2GRAY)

w, h = template.shape[::-1]

res = cv2.matchTemplate(target_rgb_gray, template, cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where(res >= threshold)
print(type(loc), len(loc))
print(type(loc[0]), len(loc[0]))
print(loc[::-1])
print(*loc[::-1]) # *作用是将列表解开成两个独立的参数
for pt in zip(*loc[::-1]):  # zip(*loc[::-1]) 等价于 zip(loc[1], loc[0])
    print(pt, (pt[0] + w, pt[1] + h))
    cv2.rectangle(target_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
 
# cv2.imwrite('res.jpg', target_rgb)
# test = cv2.imread(IMAGE_TARGET)
# cv2.imshow('original', test)
# cv2.imshow('res', target_rgb)
# cv2.waitKey(0) & 0xFF
# cv2.destroyAllWindows()


<class 'tuple'> 2
<class 'numpy.ndarray'> 3
(array([175, 175, 175], dtype=int64), array([133, 218, 303], dtype=int64))
[175 175 175] [133 218 303]
(175, 133) (190, 152)
(175, 218) (190, 237)
(175, 303) (190, 322)


In [5]:
def show_cv2_image(img):
    cv2.imshow('original', img)
    cv2.waitKey(0) & 0xFF
    cv2.destroyAllWindows()

show_cv2_image(target_rgb)



In [38]:

# *作用是将列表解开成两个独立的参数，传入函数，还有类似的有两个星号，是将字典解开成独立的元素作为形参
a = [1, 2, 3]
print(a[::-1])

a=[1,2,3]
b=[('a1', 'a2'), ('b1', 'b2'), ('c1', 'c2')]
c={'1':'a','2':'b','3':'c'}
print(a,"",*a)
print(b,"",*b)
print(b,"",*b)
print(c,"====",*c)

[3, 2, 1]
[1, 2, 3]  1 2 3
[('a1', 'a2'), ('b1', 'b2'), ('c1', 'c2')]  ('a1', 'a2') ('b1', 'b2') ('c1', 'c2')
[('a1', 'a2'), ('b1', 'b2'), ('c1', 'c2')]  ('a1', 'a2') ('b1', 'b2') ('c1', 'c2')
{'1': 'a', '2': 'b', '3': 'c'} ==== 1 2 3


In [26]:
# imread函数有两个参数，第一个参数是图片路径，第二个参数表示读取图片的形式，有三种：
# cv2.IMREAD_COLOR：加载彩色图片，这个是默认参数，可以直接写1。
# cv2.IMREAD_GRAYSCALE：以灰度模式加载图片，可以直接写0。
# cv2.IMREAD_UNCHANGED：包括alpha，可以直接写-1

template = cv2.imread(IMAGE_TARGET)   # 
print(template.shape)
template = pil2cv(Image.open(IMAGE_TARGET))
print(template.shape)



(418, 688, 3)
(418, 688, 3)


In [31]:
# 比较两个库的性能差别
import pyautogui
import time

# template = cv2.imread(IMAGE_TEMPLATE_MUL, 0)  # numpy.ndarray
# template = cv2.imread(IMAGE_TEMPLATE_ONE, 0)
# target_rgb = cv2.imread(IMAGE_TARGET)
# target_rgb_gray = cv2.cvtColor(target_rgb, cv2.COLOR_BGR2GRAY)
img_template = Image.open(IMAGE_TEMPLATE_ONE)
img_target = Image.open(IMAGE_TARGET)
template = pil2cv(img_template)
target_rgb_gray = pil2cv(img_target)
img_template = cv2pil(template)
img_target = cv2pil(target_rgb_gray)

print(template.shape, target_rgb_gray.shape)

def locate_im_by_cv2(template, target_rgb_gray):
    h, w = target_rgb_gray.shape[0], target_rgb_gray.shape[1]
    res = cv2.matchTemplate(target_rgb_gray, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    loc = np.where(res >= threshold)
    for pt in zip(*loc[::-1]):  # zip(*loc[::-1]) 等价于 zip(loc[1], loc[0])
        # print(pt, (pt[0] + w, pt[1] + h))
        cv2.rectangle(target_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
        return (pt[0], pt[1], w, h)


def locate_im_exact(template, target):
    '''通过坐标来获取截图并查看图片是否存在'''
    try:
        loc = pyautogui.locate(template, target, confidence=0.8)
        return loc
    except Exception as error:
        self.display_msg('截图比对失败：' + str(error))
        return None

time1 = []
time2 = []
for i in range(5):
    start = time.time()
    print(locate_im_by_cv2(template, target_rgb_gray))
    end = time.time()
    time1.append(end - start)

    start = time.time()
    print(locate_im_exact(img_template, img_target))
    end = time.time()
    time2.append(end - start)
print(time1)
print(time2)


(20, 19) (418, 688)
(153, 303, 688, 418)
Box(left=153, top=303, width=19, height=20)
(153, 303, 688, 418)
Box(left=153, top=303, width=19, height=20)
(153, 303, 688, 418)
Box(left=153, top=303, width=19, height=20)
(153, 303, 688, 418)
Box(left=153, top=303, width=19, height=20)
(153, 303, 688, 418)
Box(left=153, top=303, width=19, height=20)
[0.01199197769165039, 0.011992692947387695, 0.011992454528808594, 0.013991355895996094, 0.01299142837524414]
[0.03598213195800781, 0.029982805252075195, 0.030983448028564453, 0.028983592987060547, 0.0359807014465332]
